# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Thái Đan Sâm
- MSSV: 21127414
- Lớp: 21CLC01

# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây
#1a
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y
        return self

    def get_params(self):
        return self.w

    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

def MAE(y_true, predictions):
    return np.mean(np.abs(y_true - predictions))

def pre_Process(X):
    return np.hstack((X,np.ones((X.shape[0],1))))


In [4]:
#1b
def k_fold_cross_validation(x_value, y_value, feature, k=5):
    fold_mae_scores = []

    fold_size = len(x_value) // k

    for fold in range(k):
        start_idx = fold * fold_size
        end_idx = (fold + 1) * fold_size
        
        X_fold_train = x_value.iloc[np.r_[:start_idx, end_idx:]].loc[:, [feature]]
        y_fold_train = y_value.iloc[np.r_[:start_idx, end_idx:]]
        
        X_fold_val = x_value.iloc[start_idx:end_idx].loc[:, [feature]]
        y_fold_val = y_value.iloc[start_idx:end_idx]
        
        # Train a linear regression model on the fold
        X_fold_train_with_bias = np.hstack((X_fold_train.to_numpy(), np.ones((X_fold_train.shape[0], 1))))
        weights = np.linalg.inv(X_fold_train_with_bias.T @ X_fold_train_with_bias) @ X_fold_train_with_bias.T @ y_fold_train.to_numpy()
        
        # Make predictions on the validation set
        X_fold_val_with_bias = np.hstack((X_fold_val.to_numpy(), np.ones((X_fold_val.shape[0], 1))))
        predictions = X_fold_val_with_bias @ weights
        
        # Calculate Mean Absolute Error for the fold
        fold_mae_scores.append(MAE(y_fold_val.to_numpy(), predictions))
    
    mean_mae = np.mean(fold_mae_scores)
    return mean_mae

def find_best_personality_feature(x_value, y_value, personality_features, k=5):
    mae_scores = {}

    for feature in personality_features:
        mean_mae = k_fold_cross_validation(x_value, y_value, feature, k)
        mae_scores[feature] = mean_mae

    return mae_scores

# Xáo trộn lại dữ liệu
def shuffle_data(X_value, y_value, random_state=42):
    # Combine X_train and y_train for shuffling
    combined_data = pd.concat([X_value, y_value], axis=1)
    
    # Shuffle the combined data
    shuffled_data = combined_data.sample(frac=1, random_state=random_state)
    
    # Split shuffled data back into X_train and y_train
    X_value_shuffled = shuffled_data.drop(columns=['Salary'])
    y_value_shuffled = shuffled_data['Salary']
    
    return X_value_shuffled, y_value_shuffled


In [5]:
X_train1=X_train.iloc[:,:11].to_numpy()
X_test1 = X_test.iloc[:,:11].to_numpy()
y_train1=y_train.to_numpy()
y_test1 = y_test.to_numpy()

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [6]:
# Phần code cho yêu cầu 1a
X_1a=pre_Process(X_train1)
reg=OLSLinearRegression().fit(X_1a,y_train1)
np.round(reg.get_params(),3)

array([-23183.33 ,    702.767,   1259.019, -99570.608,  18369.962,
         1297.532,  -8836.727,    141.76 ,    145.742,    114.643,
        34955.75 ,  49248.09 ])

In [7]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
predict =reg.predict(pre_Process(X_test1))
print('MAE:', MAE(y_test1,predict))

MAE: 105052.52978823186


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -23183.330x_1 + 702.767x_2 + 1259.019x_3 + -99570.608x_4 + 18369.962x_5 + 1297.532x_6 + -8836.727x_7 + 141.760x_8 
+ 145.742x_9 + 114.643x_{10} + 34955.750x_{11} + 49248.090x_{12}$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [8]:
# Phần code cho yêu cầu 1b
X_1b = X_train.iloc[:, 18:]
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
X_train_shuffled, y_train_shuffled = shuffle_data(X_train, y_train)
mae_scores = find_best_personality_feature(X_train_shuffled, y_train_shuffled, X_1b, k = 5)

best_feature = min(mae_scores, key=mae_scores.get)
print(f"The best personality feature: {best_feature}")

best_mae = mae_scores[best_feature]
print(f"The value MAE of the best feature: {best_mae}")

prediction = pd.DataFrame(mae_scores.items(), columns=['Feature', 'MAE'])
prediction

The best personality feature: nueroticism
The value MAE of the best feature: 123067.98457443409


,Feature,MAE
0,conscientiousness,124016.187460
1,agreeableness,123098.142270
2,extraversion,123668.596905
3,nueroticism,123067.984574
4,openess_to_experience,123687.015610


In [9]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_feature_column = X_train_shuffled[best_feature].to_numpy().reshape(-1, 1) 

X_train_bias=np.hstack((best_feature_column,np.ones((best_feature_column.shape[0],1))))

# Train the model on the entire training dataset with the best feature
weights_best = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train_shuffled.to_numpy()

# Evaluate the model on the training dataset using mean absolute error
predictions_train = X_train_bias @ weights_best
mae_train = MAE(y_train_shuffled, predictions_train)

print("The value MAE of the Training Data:", mae_train)

# Print the weights (coefficients) of the trained model
print("Coeficients and interception:", ", ".join(["{:.3f}".format(s) for s in weights_best]))

The value MAE of the Training Data: 123388.23814751685
Coeficients and interception: -16021.494, 304647.553


In [10]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
X_test_shuffled, y_test_shuffled = shuffle_data(X_test, y_test)
best_feature_column_test = X_test_shuffled[best_feature].to_numpy().reshape(-1, 1)

X_test_bias=np.hstack((best_feature_column_test,np.ones((best_feature_column_test.shape[0],1))))

# Make predictions on the test dataset using the trained model
predictions_test = X_test_bias @ weights_best

# Calculate Mean Absolute Error on the test dataset
mae_test = MAE(y_test_shuffled, predictions_test)
print("The value MAE of the Test Data:", mae_test)

The value MAE of the Test Data: 119361.91739987815


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -16021.494*nueroticism + 304647.553$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [11]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

X_1c=X_train[['English','Logical','Quant']]

X_train_shuffled, y_train_shuffled = shuffle_data(X_train, y_train)
mae_scores = find_best_personality_feature(X_train_shuffled, y_train_shuffled, X_1c, k = 5)

best_feature = min(mae_scores, key=mae_scores.get)
print(f"The best personality feature: {best_feature}")

best_mae = mae_scores[best_feature]
print(f"The value MAE of the best feature: {best_mae}")

prediction = pd.DataFrame(mae_scores.items(), columns=['Feature', 'MAE'])
prediction


The best personality feature: Quant
The value MAE of the best feature: 116865.81719922007


,Feature,MAE
0,English,120121.858199
1,Logical,119460.779750
2,Quant,116865.817199


In [12]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_feature_column = X_train_shuffled[best_feature].to_numpy().reshape(-1, 1) 

X_train_bias=np.hstack((best_feature_column,np.ones((best_feature_column.shape[0],1))))

# Train the model on the entire training dataset with the best feature
weights_best = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train_shuffled.to_numpy()

# Evaluate the model on the training dataset using mean absolute error
predictions_train = X_train_bias @ weights_best
mae_train = MAE(y_train_shuffled, predictions_train)

print("The value MAE of the Training Data:", mae_train)

# Print the weights (coefficients) of the trained model
print("Coeficients and interception:", ", ".join(["{:.3f}".format(s) for s in weights_best]))

The value MAE of the Training Data: 117166.19656080763
Coeficients and interception: 368.852, 117759.729


In [13]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
X_test_shuffled, y_test_shuffled = shuffle_data(X_test, y_test)
best_feature_column_test = X_test_shuffled[best_feature].to_numpy().reshape(-1, 1)

X_test_bias=np.hstack((best_feature_column_test,np.ones((best_feature_column_test.shape[0],1))))

# Make predictions on the test dataset using the trained model
predictions_test = X_test_bias @ weights_best

# Calculate Mean Absolute Error on the test dataset
mae_test = MAE(y_test_shuffled, predictions_test)
print("The value MAE of the Test Data:", mae_test)

The value MAE of the Test Data: 108814.05968837178


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 368.852*Quant+117759.729$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [14]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [15]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [16]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [17]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$